# 🧾 Sales & Demand Planning – Student Notebook (Moderate Difficulty)

This notebook walks you through **SEE → TREAT → VERIFY** and basic forecasting on weekly SKU sales data.

**You will:**
- Inspect and understand the raw dataset (`df_raw`)
- Fix calendar and anomaly issues to create `df_treat`
- Compute descriptive & diagnostic KPIs
- Build a simple moving-average forecast
- Make a basic replenishment decision

Wherever a transformation is non-trivial, the **exact formula** is written in markdown.
You still need to write / uncomment the code, but you don't have to invent the logic.

## 0. Imports & Dataset Loading

In [ ]:
# TODO: import pandas, numpy, matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# TODO: load df_raw from your CSV
# Example:
df_raw = pd.read_csv('https://raw.githubusercontent.com/saikisri97/17_Hof_Lecture_Code_Pingo/refs/heads/main/Supply_Chain_Analytics/data/sales_demand_planning.csv')
df_raw.head()

,Week,SKU,Sales_Units,Is_Promo,Price
0,2023-05-14,SKU_B,213,0,19.49
1,2023-03-19,SKU_A,157,0,14.99
2,2024-09-29,SKU_B,-25,1,14.49
3,2024-01-21,SKU_B,197,0,19.49
4,2023-11-19,SKU_B,187,0,19.49


## 0.1 Helper Functions (Already Implemented for You)

In [2]:
def missing_report(df):
    """Print count of missing values per column."""
    print(df.isna().sum())

def check_week_structure(df):
    """Check dtypes and duplicate (Week, SKU) rows."""
    print('dtypes:\n', df.dtypes)
    if 'Week' in df.columns and 'SKU' in df.columns:
        dup = df.duplicated(subset=['Week', 'SKU']).sum()
        print('Duplicate (Week, SKU) rows:', dup)

def check_anomalies(df):
    """Check negative sales and extreme promo spikes."""
    if 'Sales_Units' in df.columns:
        print('\nNegative sales rows:')
        print(df[df['Sales_Units'] < 0])
    if 'Is_Promo' in df.columns and 'Sales_Units' in df.columns:
        print('\nTop promo spikes:')
        print(df[df['Is_Promo'] == 1].sort_values('Sales_Units', ascending=False).head())

def step_check(step_name, df):
    print(f'--- {step_name} ---')
    print(df.head())


## STEP 1A – SEE: Structural Checks

Use:
- `df_raw.dtypes`
- `df_raw.shape`
- `missing_report(df_raw)`
- `check_week_structure(df_raw)`

Goal: detect wrong dtypes, missing columns, duplicate (Week, SKU) rows.

In [3]:
# TODO: run structural checks on df_raw



Week            object
SKU             object
Sales_Units      int64
Is_Promo         int64
Price          float64
dtype: object
(206, 5)
Week           0
SKU            0
Sales_Units    0
Is_Promo       0
Price          0
dtype: int64
dtypes:
 Week            object
SKU             object
Sales_Units      int64
Is_Promo         int64
Price          float64
dtype: object
Duplicate (Week, SKU) rows: 4


## STEP 1B – SEE: Anomaly Checks

Check for:
- Negative `Sales_Units`
- Extreme promo spikes
- Suspicious `Price` values

Useful code patterns:
```python
df_raw['Sales_Units'].describe()
df_raw[df_raw['Sales_Units'] < 0]
df_raw[df_raw['Is_Promo'] == 1].sort_values('Sales_Units', ascending=False).head()
df_raw['Price'].describe()
```

In [4]:
# TODO: run anomaly checks


check_anomalies(df_raw)


count     206.000000
mean      189.961165
std       104.782229
min       -25.000000
25%       149.250000
50%       182.000000
75%       207.000000
max      1275.000000
Name: Sales_Units, dtype: float64
          Week    SKU  Sales_Units  Is_Promo  Price
2   2024-09-29  SKU_B          -25         1  14.49
30  2024-01-14  SKU_A           -5         0  14.99
33  2023-04-23  SKU_B           -5         0  19.49
           Week    SKU  Sales_Units  Is_Promo  Price
130  2023-10-29  SKU_B         1275         1  14.49
194  2024-07-07  SKU_A          975         1   9.99
29   2024-03-24  SKU_B          316         1  14.49
202  2023-02-19  SKU_B          304         1  14.49
123  2024-05-19  SKU_B          303         1  14.49
count    206.000000
mean      16.468155
std        2.854309
min        9.990000
25%       14.990000
50%       14.990000
75%       19.490000
max       19.490000
Name: Price, dtype: float64

Negative sales rows:
          Week    SKU  Sales_Units  Is_Promo  Price
2   2024-0

## STEP 1C – SEE Summary
Summarise what is wrong with the raw data.

In [5]:
# TODO: summarise main data issues you saw
see_summary = '''
- Week may be stored as string and can contain invalid patterns
- Some (Week, SKU) combinations may be duplicated
- There may be negative Sales_Units and big promo spikes
- Prices may not always match promo/non-promo expectations
'''
print(see_summary)



- Week may be stored as string and can contain invalid patterns
- Some (Week, SKU) combinations may be duplicated
- There may be negative Sales_Units and big promo spikes
- Prices may not always match promo/non-promo expectations



## STEP 2A – TREAT: Fix Calendar & Missing Weeks (Guided)

Goal: create a clean weekly time series `df_treat`.

Follow these **four mini-fixes**, using the exact formulas.

### 🧩 FIX 1 — Convert `Week` → `Week_dt` (datetime)
```python
df_clean = df_raw.copy()
df_clean['Week_dt'] = pd.to_datetime(df_clean['Week'], errors='coerce')
df_clean = df_clean.dropna(subset=['Week_dt'])
```

### 🧩 FIX 2 — Collapse duplicate (SKU, Week_dt) rows
```python
df_clean = df_clean.groupby(['SKU','Week_dt'], as_index=False).agg({
    'Sales_Units': 'sum',
    'Is_Promo': 'max',
    'Price': 'mean'
})
```

### 🧩 FIX 3 — Create the full weekly calendar
```python
full_weeks = pd.date_range(
    df_clean['Week_dt'].min().normalize(),
    df_clean['Week_dt'].max().normalize(),
    freq='W-SUN'
)
```

### 🧩 FIX 4 — Reindex each SKU to the full calendar
```python
df_list = []
for sku, sub in df_clean.groupby('SKU'):
    sub = sub.set_index('Week_dt').reindex(full_weeks)
    sub['SKU'] = sku
    df_list.append(sub)

df_treat = pd.concat(df_list).reset_index().rename(columns={'index':'Week_dt'})
df_treat['Week'] = df_treat['Week_dt'].dt.strftime('%Y-%m-%d')
```


In [11]:
# STEP 2A – implement the 4 fixes using the formulas above
# 1) Week → Week_dt
df_clean = df_raw.copy()
df_clean['Week_dt'] = 
df_clean = df_clean.dropna(subset=['Week_dt'])




--- 2A_calendar_fixed ---
     Week_dt    SKU  Sales_Units  Is_Promo  Price        Week
0 2023-01-01  SKU_A        145.0       0.0  14.99  2023-01-01
1 2023-01-08  SKU_A        141.0       0.0  14.99  2023-01-08
2 2023-01-15  SKU_A        210.0       1.0   9.99  2023-01-15
3 2023-01-22  SKU_A        163.0       0.0  14.99  2023-01-22
4 2023-01-29  SKU_A        148.0       0.0  14.99  2023-01-29


In [ ]:
# 2) Collapse duplicates per SKU & Week_dt
df_clean = df_clean.groupby(['SKU','Week_dt'], as_index=False).agg({
    'Sales_Units': '',
   'Is_Promo': '',
    'Price': ''
 })



In [ ]:
# 3) Build full weekly calendar
full_weeks = pd.date_range(
  
)



In [ ]:
# 4) Reindex per SKU
df_list = []




In [ ]:
df_treat = pd.concat(df_list).reset_index().rename(columns={'index':'Week_dt'})
df_treat['Week'] = df_treat['Week_dt'].dt.strftime('%Y-%m-%d')

step_check('2A_calendar_fixed', df_treat)

## STEP 2B – TREAT: Fix Anomalies

Now clean value problems in `df_treat`.

### 🧩 FIX A — Replace negative sales with 0
```python
df_treat['Sales_Units'] = df_treat['Sales_Units'].fillna(0)
df_treat.loc[df_treat['Sales_Units'] < 0, 'Sales_Units'] = 0
```

### 🧩 FIX B — Cap extreme promo spikes per SKU
```python
for sku, sub in df_treat.groupby('SKU'):
    mask = (df_treat['SKU'] == sku) & (df_treat['Is_Promo'] == 1)
    cap = sub[sub['Is_Promo'] == 1]['Sales_Units'].quantile(0.99)
    df_treat.loc[mask & (df_treat['Sales_Units'] > cap), 'Sales_Units'] = cap
```

### 🧩 FIX C — Fix prices using forward/backward fill
```python
df_treat['Price'] = df_treat.groupby('SKU')['Price'].ffill().bfill()
```

In [12]:
# STEP 2B – apply anomaly fixes from the formulas above
df_treat['Sales_Units'] = 

for sku, sub in df_treat.groupby('SKU'):
     

df_treat['Price'] = df_treat.groupby('SKU')['Price'].ffill().bfill()


step_check('2B_anomalies_fixed', df_treat)


--- 2B_anomalies_fixed ---
     Week_dt    SKU  Sales_Units  Is_Promo  Price        Week
0 2023-01-01  SKU_A        145.0       0.0  14.99  2023-01-01
1 2023-01-08  SKU_A        141.0       0.0  14.99  2023-01-08
2 2023-01-15  SKU_A        210.0       1.0   9.99  2023-01-15
3 2023-01-22  SKU_A        163.0       0.0  14.99  2023-01-22
4 2023-01-29  SKU_A        148.0       0.0  14.99  2023-01-29


## STEP 2C – VERIFY after TREAT

Re-run your SEE helpers on `df_treat`.

Use:
```python
missing_report(df_treat)
check_week_structure(df_treat)
check_anomalies(df_treat)
```

In [13]:
# STEP 2C – VERIFY


Week_dt        0
SKU            0
Sales_Units    0
Is_Promo       7
Price          0
Week           0
dtype: int64
dtypes:
 Week_dt        datetime64[ns]
SKU                    object
Sales_Units           float64
Is_Promo              float64
Price                 float64
Week                   object
dtype: object
Duplicate (Week, SKU) rows: 0

Negative sales rows:
Empty DataFrame
Columns: [Week_dt, SKU, Sales_Units, Is_Promo, Price, Week]
Index: []

Top promo spikes:
       Week_dt    SKU  Sales_Units  Is_Promo  Price        Week
147 2023-10-29  SKU_B      1140.74       1.0  14.49  2023-10-29
79  2024-07-07  SKU_A       874.34       1.0   9.99  2024-07-07
168 2024-03-24  SKU_B       316.00       1.0  14.49  2024-03-24
111 2023-02-19  SKU_B       304.00       1.0  14.49  2023-02-19
176 2024-05-19  SKU_B       303.00       1.0  14.49  2024-05-19


## STEP 3A – Descriptive Analytics

Compute basic stats per SKU:
```python
df_treat.groupby('SKU')['Sales_Units'].agg(['mean','std','min','max'])
```

Compare promo vs non-promo averages:
```python
promo_vs_non = df_treat.groupby(['SKU','Is_Promo'])['Sales_Units'].mean().unstack()
```

In [14]:
# TODO: descriptive KPIs
desc_stats = df_treat.groupby('SKU')['Sales_Units'].agg
print(desc_stats)

promo_vs_non = df_treat.groupby(['SKU','Is_Promo'])
print(promo_vs_non)


             mean         std  min      max
SKU                                        
SKU_A  163.955192   80.474425  0.0   874.34
SKU_B  208.324423  112.392169  0.0  1140.74
Is_Promo         0.0      1.0
SKU                          
SKU_A     151.793103  256.356
SKU_B     201.047619  318.516


## STEP 3B – Diagnostic Analytics

Compute promo uplift (%):
```python
uplift = (promo_vs_non[1] - promo_vs_non[0]) / promo_vs_non[0] * 100
```

In [15]:
# TODO: compute promo uplift
uplift = 
print('Promo uplift (%):')
print(uplift)


Promo uplift (%):
SKU
SKU_A    68.885143
SKU_B    58.428138
dtype: float64


## STEP 3C – Predictive Analytics (3-week Moving Average)

Compute a 3-week simple moving average per SKU:
```python
df_ma_list = []
for sku, sub in df_treat.groupby('SKU'):
    sub = sub.sort_values('Week_dt').copy()
    sub['SMA_3'] = sub['Sales_Units'].rolling(window=3).mean()
    df_ma_list.append(sub)

df_forecast_all = pd.concat(df_ma_list)
```

In [21]:
df_ma_list = []
for sku, sub in df_treat.groupby('SKU'):


df_forecast_all = pd.concat(df_ma_list)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def bias(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(y_pred - y_true)

for sku, sub in df_forecast_all.groupby('SKU'):
    eval_sub = sub.dropna(subset=['SMA_3']).tail(20)
    m = mape(eval_sub['Sales_Units'], eval_sub['SMA_3'])
    b = bias(eval_sub['Sales_Units'], eval_sub['SMA_3'])
    print(f'SKU {sku}: MAPE={m:.1f}%, Bias={b:.2f}')


SKU SKU_A: MAPE=11.4%, Bias=-0.17
SKU SKU_B: MAPE=17.6%, Bias=-3.77


## STEP 4A – Forecast Next Week

Use the latest SMA_3 value for each SKU as next-week forecast:
```python
next_week_forecast = {}
for sku, sub in df_forecast_all.groupby('SKU'):
    latest = sub.dropna(subset=['SMA_3']).sort_values('Week_dt').iloc[-1]
    next_week_forecast[sku] = latest['SMA_3']
```

In [18]:
# TODO: create next_week_forecast dict
next_week_forecast = {}
for sku, sub in df_forecast_all.groupby('SKU'):

print(next_week_forecast)


{'SKU_A': 150.33333333333334, 'SKU_B': 203.33333333333334}


## STEP 4B – Prescriptive: Replenishment Decision

Assume:
- Service level factor: `z = 1.65` (≈95%)
- Safety stock formula: `Safety_Stock = z * weekly_std`
- Reorder point: `ROP = forecast + Safety_Stock`
- Use a simple assumption for `current_inventory` (e.g. equal to forecast).

Example pattern:
```python
service_level_z = 1.65
replenishment_plan = {}
for sku, sub in df_treat.groupby('SKU'):
    sub = sub.sort_values('Week_dt')
    weekly_std = sub['Sales_Units'].tail(26).std()
    forecast = next_week_forecast[sku]
    safety_stock = service_level_z * weekly_std
    rop = forecast + safety_stock
    current_inventory = forecast
    order_qty = max(0, rop - current_inventory)
```

In [31]:
# TODO: compute safety stock, ROP, and order quantity per SKU
service_level_z = 1.65
replenishment_plan = {}

for sku, sub in df_treat.groupby('SKU'):

    replenishment_plan[sku] = {
        'forecast': forecast,
        'weekly_std': weekly_std,
        'safety_stock': safety_stock,
        'ROP': rop,
        'current_inventory': current_inventory,
        'order_quantity': order_qty,
       
    }
replenishment_plan


{'SKU_A': {'forecast': 150.33333333333334,
  'weekly_std': 144.2608863776371,
  'safety_stock': 238.0304625231012,
  'ROP': 388.36379585643454,
  'current_inventory': 150.33333333333334,
  'order_quantity': 238.0304625231012},
 'SKU_B': {'forecast': 203.33333333333334,
  'weekly_std': 77.8089672506671,
  'safety_stock': 128.3847959636007,
  'ROP': 331.7181292969341,
  'current_inventory': 203.33333333333334,
  'order_quantity': 128.38479596360074}}